In [1]:
import os

In [2]:
os.listdir('../')

['lib', 'input', 'working']

In [4]:
pip install calmap

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from plotnine import *
import calmap
import plotly.express as px
import folium

cdr = ['#393e46', '#ff2e63', '#30e3ca'] # grey - red - blue
idr = ['#f8b400', '#ff2e63', '#30e3ca'] # yellow - red - blue

In [8]:
os.listdir('../input/corona-virus-report/covid_19_clean_complete.csv')

['covid_19_clean_complete.csv']

In [17]:
full_table=pd.read_csv('../input/corona-virus-report/covid_19_clean_complete.csv',
                      parse_dates=['Date'])

In [32]:
full_table.head(20)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,Anhui,China,31.82570,117.2264,2020-01-22,1,0,0
1,Beijing,China,40.18240,116.4142,2020-01-22,14,0,0
2,Chongqing,China,30.05720,107.8740,2020-01-22,6,0,0
3,Fujian,China,26.07890,117.9874,2020-01-22,1,0,0
4,Gansu,China,36.06110,103.8343,2020-01-22,0,0,0
5,Guangdong,China,23.34170,113.4244,2020-01-22,26,0,0
6,Guangxi,China,23.82980,108.7881,2020-01-22,2,0,0
7,Guizhou,China,26.81540,106.8748,2020-01-22,1,0,0
8,Hainan,China,19.19590,109.7453,2020-01-22,4,0,0
9,Hebei,China,38.04280,114.5149,2020-01-22,1,0,0


In [19]:
full_table['Country/Region']=full_table['Country/Region'].replace('Mainland China','China')

In [23]:
full_table['Province/State']=full_table['Province/State'].fillna('NA')

In [27]:
ship=full_table[full_table['Province/State']=='Diamond Princess cruise ship']
full_table=full_table[full_table['Province/State']!='Diamond Princess cruise ship']
china=full_table[full_table['Country/Region']=='China']
row=full_table[full_table['Country/Region']!='China']

full_latest=full_table[full_table['Date']==max(full_table['Date'])].reset_index()
china_latest=full_latest[full_latest['Country/Region']=='China']
row_latest=full_latest[full_latest['Country/Region']!='China']

full_latest_grouped=full_latest.groupby('Country/Region')['Confirmed','Deaths','Recovered'].sum().reset_index()
china_latest_grouped=china_latest.groupby('Province/State')['Confirmed','Deaths','Recovered'].sum().reset_index()
row_latest_grouped=row_latest.groupby('Country/Region')['Confirmed','Deaths','Recovered'].sum().reset_index()

In [31]:
temp=full_latest.groupby(['Country/Region','Province/State'])['Confirmed','Deaths','Recovered'].sum()
temp.style.background_gradient(cmap='Pastel1_r')

In [43]:
temp=full_table.groupby('Date')['Confirmed','Deaths','Recovered'].sum()
temp=temp.reset_index()
temp=temp.sort_values('Date',ascending=False)
temp.head(1).style.background_gradient(cmap='Pastel1')

,Date,Confirmed,Deaths,Recovered
42,2020-03-04 00:00:00,94418,3248,51161


In [48]:
#temp_f=full_latest_grouped[['Country/Region','Confirmed','Deaths','Recovered']]
temp_f=full_latest_grouped.sort_values(by='Confirmed',ascending=False)
temp_f=temp_f.reset_index(drop=True)
temp_f.style.background_gradient(cmap='Pastel1_r')

,Country/Region,Confirmed,Deaths,Recovered
0,China,80271,2981,49955
1,South Korea,5621,35,41
2,Italy,3089,107,276
3,Iran,2922,92,552
4,Japan,331,6,43
5,France,285,4,12
6,Germany,262,0,16
7,Spain,222,2,2
8,US,153,11,8
9,Singapore,110,0,78


In [58]:
temp_flg=full_latest_grouped[['Country/Region','Deaths']]
temp_flg=temp_flg.sort_values(by='Deaths',ascending=False)
temp_flg=temp_flg.reset_index(drop=True)
temp_flg=temp_flg[temp_flg['Deaths']>0]
temp_flg.style.background_gradient(cmap='Reds')

,Country/Region,Deaths
0,China,2981
1,Italy,107
2,Iran,92
3,South Korea,35
4,US,11
5,Japan,6
6,France,4
7,Spain,2
8,Hong Kong,2
9,Australia,2


In [63]:
temp=full_latest_grouped[full_latest_grouped['Recovered']==0]
temp=temp.sort_values(by='Confirmed',ascending=False)
temp=temp.reset_index(drop=True)
temp.style.background_gradient(cmap='Reds')

,Country/Region,Confirmed,Deaths,Recovered
0,Norway,56,0,0
1,Kuwait,56,0,0
2,Bahrain,52,0,0
3,Netherlands,38,0,0
4,Iraq,35,2,0
5,Sweden,35,0,0
6,Austria,29,0,0
7,Iceland,26,0,0
8,San Marino,16,1,0
9,Algeria,12,0,0


In [64]:
temp=row_latest_grouped[row_latest_grouped['Confirmed']==row_latest_grouped['Deaths']]
temp=temp.sort_values('Confirmed',ascending=False)
temp=temp.reset_index(drop=True)
temp.style.background_gradient(cmap='Reds')

,Country/Region,Confirmed,Deaths,Recovered


In [77]:
fig=px.bar(full_latest_grouped.sort_values('Confirmed',ascending=False),
          x='Country/Region',y='Confirmed',color='Country/Region',
          log_y=True,template='ggplot2',title='Confirmed Cases')
fig.show()


In [76]:
fig=px.bar(full_latest_grouped[['Country/Region','Deaths']].sort_values('Deaths',ascending=False)[full_latest_grouped['Deaths']>20].reset_index(drop=True),
          x='Country/Region',y='Deaths',template='ggplot2',color='Country/Region',
          log_y=True,title='Deaths')
fig.show()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [87]:
m=folium.Map(location=[0,0],titles='cartodbpositron',
            min_zoom=1,max_zoom=7,zoom_start=1)

for i in range(0,len(full_latest)):
    folium.Circle(
    location=[full_latest.iloc[i]['Lat'],full_latest.iloc[i]['Long']],
    color='crimson',
    tooltip=
    '<li><bold>Country : '+str(full_latest.iloc[i]['Country/Region'])+
    '<li><bold>Province : '+str(full_latest.iloc[i]['Province/State'])+
    '<li><bold>Confirmed : '+str(full_latest.iloc[i]['Confirmed'])+
    '<li><bold>Deaths : '+str(full_latest.iloc[i]['Deaths'])+
    '<li><bold>Recovered : '+str(full_latest.iloc[i]['Recovered']),
    radius=np.log(full_latest.iloc[i]['Confirmed'])).add_to(m)
m


SyntaxError: invalid syntax (<ipython-input-87-0261880c5e35>, line 15)